In [6]:
from openai import OpenAI
import xml.etree.ElementTree as ET
from tqdm import tqdm

xml_file_path = '../data/Task 2/NBER_working_papers.xml'
tree = ET.parse(xml_file_path)
root = tree.getroot()

title = root.find('.//atl').text #find returns first instance of title, abstract
abstract = root.find('.//ab').text

In [7]:
import pandas as pd
import xml.etree.ElementTree as ET

# Extract relevant data
data = []
for rec in root.findall('rec'):
    header = rec.find('header')
    result_id = rec.get('resultID')
    
    # Extract header attributes
    short_db_name = header.get('shortDbName')
    long_db_name = header.get('longDbName')
    ui_term = header.get('uiTerm')

    # Extract control info
    control_info = header.find('controlInfo')
    pubinfo = control_info.find('pubinfo')
    artinfo = control_info.find('artinfo')
    display_info = header.find('displayInfo')
    
    # Extracting various fields with error handling
    article_data = {
        'resultID': result_id,
        'shortDbName': short_db_name,
        'longDbName': long_db_name,
        'uiTerm': ui_term,
        'jtl': control_info.find('./jinfo/jtl').text if control_info.find('./jinfo/jtl') is not None else None,
        'issn': control_info.find('./jinfo/issn').text if control_info.find('./jinfo/issn') is not None else None,
        'publication_date': pubinfo.find('dt').text if pubinfo.find('dt') is not None else None,
        'volume': pubinfo.find('vid').text if pubinfo.find('vid') is not None else None,
        'issue': pubinfo.find('iid').text if pubinfo.find('iid') is not None else None,
        'doi': artinfo.find('ui').text if artinfo.find('ui') is not None else None,
        'ppct': artinfo.find('ppct').text if artinfo.find('ppct') is not None else None,
        'pages': artinfo.find('pages').text if artinfo.find('pages') is not None else None,
        'title': artinfo.find('./tig/atl').text if artinfo.find('./tig/atl') is not None else None,
        'authors': ', '.join([au.text for au in artinfo.findall('./aug/au')]) if artinfo.findall('./aug/au') else None,
        'affiliations': ', '.join([affil.text for affil in artinfo.findall('./aug/affil')]) if artinfo.findall('./aug/affil') else None,
        'subjects': ', '.join([su.text for su in artinfo.findall('./su')]) if artinfo.findall('./su') else None,
        'abstract': artinfo.find('ab').text if artinfo.find('ab') is not None else None,
        'publication_type': artinfo.find('pubtype').text if artinfo.find('pubtype') is not None else None,
        'language': control_info.find('./language').text if control_info.find('./language') is not None else None,
        'url': display_info.find('./pLink/url').text if display_info.find('./pLink/url') is not None else None
    }
    
    data.append(article_data)

# Create DataFrame
df = pd.DataFrame(data)

In [8]:
df

,resultID,shortDbName,longDbName,uiTerm,jtl,issn,publication_date,volume,issue,doi,ppct,pages,title,authors,affiliations,subjects,abstract,publication_type,language,url
0,1,ecn,EconLit,2098011,None,None,20240101,None,None,None,None,None,A Supply Curve for Forest-Based CO2 Removal,"Franklin, Sergio L., Jr., Pindyck, Robert S.","Unlisted, Unlisted",Mathematical Methods; Programming Models; Math...,Forestation is viewed as an important means of...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
1,2,ecn,EconLit,2098010,None,None,20240101,None,None,None,None,None,New Area- and Population-based Geographic Cros...,"Ferrara, Andreas, Testa, Patrick A., Zhou, Liyang","Unlisted, Unlisted, Unlisted",Development of the Discipline: Historiographic...,"In applied historical research, geographic uni...",Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
2,3,ecn,EconLit,2098009,None,None,20240101,None,None,None,None,None,Decarbonizing Aviation: Cash-for-Clunkers in t...,"Brueckner, Jan K., Kahn, Matthew E., Nickelsbu...","Unlisted, Unlisted, Unlisted",Climate; Natural Disasters and Their Managemen...,The durability of the transportation capital s...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
3,4,ecn,EconLit,2098008,None,None,20240101,None,None,None,None,None,The Multigenerational Impact of Children and C...,"Karademir, Sencer, Laliberte, Jean-William P.,...","Unlisted, Unlisted, Unlisted",Fiscal Policies and Behavior of Economic Agent...,This paper examines the multigenerational impa...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
4,5,ecn,EconLit,2098007,None,None,20240101,None,None,None,None,None,Inequality Within Countries is Falling: Underr...,"Pinkovskiy, Maxim, Sala-i-Martin, Xavier, Chat...","Unlisted, Unlisted, Unlisted, Unlisted","Survey Methods; Sampling Methods, Personal Inc...",Household surveys suffer from persistent and g...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2576,2577,ecn,EconLit,1965085,None,None,20220101,None,None,None,None,None,Global Innovation and Knowledge Diffusion,"Lind, Nelson, Ramondo, Natalia","Unlisted, Unlisted","Trade: General, Innovation; Research and Devel...",We develop a Ricardian model of trade in which...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
2577,2578,ecn,EconLit,1965084,None,None,20220101,None,None,None,None,None,"Macroeconomic Research, Present and Past","Glandon, Philip J., Kuttner, Kenneth, Mazumder...","Unlisted, Unlisted, Unlisted, Unlisted",Role of Economics; Role of Economists; Market ...,How is macroeconomic research conducted and wh...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
2578,2579,ecn,EconLit,1965083,None,None,20220101,None,None,None,None,None,Revealing Corruption: Firm and Worker Level Ev...,"Colonnelli, Emanuele, Lagaras, Spyridon, Ponti...","Unlisted, Unlisted, Unlisted, Unlisted, Unlisted",Bureaucracy; Administrative Processes in Publi...,We study how the disclosure of corrupt practic...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...
2579,2580,ecn,EconLit,1965082,None,None,20220101,None,None,None,None,None,Debt as Safe Asset,"Brunnermeier, Markus K., Merkel, Sebastian A.,...","Unlisted, Unlisted, Unlisted","Financial Markets and the Macroeconomy, Portfo...",The price of a safe asset reflects not only th...,Working Paper,English,https://search.ebscohost.com/login.aspx?direct...


In [10]:
import json

# creates the api_batch_request.jsonl file in ../Task xyz/data

def create_batch_jsonl(df, file_path):
    """
    Create a .jsonl file for batching API requests.
    :param df: Pandas DataFrame containing 'title' and 'abstract' columns
    :param file_path: Path to save the output .jsonl file
    """
    with open(file_path, 'w') as f:
        for idx, row in df.iterrows():
            title = row['title']
            abstract = row['abstract']
            prompt = f""" Based on the title and abstract of the following research paper, evaluate its relevance to the topic 'linking environmental changes to health' on a scale from 0 (no relevance) to 10 (highly relevant). Please provide only a single number (from 0 to 10) as your response without any additional explanation.            Title: {title}            Abstract: {abstract}"""
    
            request_data = {
                "custom_id": f"request-{idx+1}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-3.5-turbo-0125",
                    "messages": [
                        {"role": "system", "content": "You are a helpful research assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    "temperature": 0.2, 
                    "max_tokens": 10
                }
            }
            f.write(json.dumps(request_data) + '\n') #json dumps converts a dictionary into json formatted string

create_batch_jsonl(df, '../data/Task 2/task2_api_batch_requests.jsonl')


In [ ]:
from openai import OpenAI
import json

from openai import OpenAI
from dotenv import load_dotenv
import os

env_path = "../keys.env"

load_dotenv(dotenv_path=env_path)
api_key = os.getenv("API_KEY")
project = os.getenv("PROJECT")
organization = os.getenv("ORGANIZATION")

client = OpenAI(
 organization=organization,
 project=project,
 api_key = api_key
)

client.batches.list() #when coming back after finishing the jobs, look at the list and get
# the appropriate batch_id if it wasn't saved. alternatively just save the batch_id it. 

SyncCursorPage[Batch](data=[Batch(id='batch_6709269010c481909c822d604504243d', completion_window='24h', created_at=1728652944, endpoint='/v1/chat/completions', input_file_id='file-Iox9uXJnfug2ZkaGGMogodhL', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1728654434, error_file_id=None, errors=None, expired_at=None, expires_at=1728739344, failed_at=None, finalizing_at=1728654056, in_progress_at=1728652946, metadata={'description': 'evaluating research papers for task 2'}, output_file_id='file-SYvv7R5fCire0m55jWSL52Gy', request_counts=BatchRequestCounts(completed=2581, failed=0, total=2581)), Batch(id='batch_66fabfeac3b88190ab854408af88914a', completion_window='24h', created_at=1727709162, endpoint='/v1/chat/completions', input_file_id='file-eb5DewtVre1LUxcLTkDe9bHq', object='batch', status='completed', cancelled_at=None, cancelling_at=None, completed_at=1727711140, error_file_id=None, errors=None, expired_at=None, expires_at=1727795562, failed_at=

In [ ]:

# upload batch file
batch_input_file = client.files.create(
  file=open("../data/Task 2/task2_api_batch_requests.jsonl", "rb"),
  purpose="batch"
)


In [14]:

batch_input_file_id = batch_input_file.id

# create batch file
batch = client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "evaluating research papers for task 2"
    }
)


In [8]:
#gets status
batch_id = "batch_6709269010c481909c822d604504243d" #usually must be saved (maybe in text, accessed with batch.id)

# important, .retrieve() takes the batch.id not the batch_input_file.id
client.batches.retrieve("batch_6709269010c481909c822d604504243d").status


'completed'

In [9]:

output_file_id = client.batches.retrieve(batch_id).output_file_id


In [10]:

output_file = client.files.content(output_file_id).text

output_path = '../data/Task 2/task2_batch-output.jsonl'
with open(output_path, 'w') as file:
    for line in output_file.splitlines():
        json_line = json.loads(line)
        file.write(json.dumps(json_line) + '\n')

print(f"Output has been written to {output_path}")

Output has been written to ../data/Task 2/task2_batch-output.jsonl


In [18]:
batch_input_file_id

'file-eb5DewtVre1LUxcLTkDe9bHq'

In [1]:

file_response = client.files.content("")
client.batches.retrieve(batch.id)


NameError: name 'client' is not defined

In [ ]:


client.batches.retrieve(batch.id)


In [ ]:

load_dotenv(dotenv_path=env_path)
api_key = os.getenv("API_KEY")
project = os.getenv("PROJECT")
organization = os.getenv("ORGANIZATION")



def generate_response(title, abstract):
    prompt = f"""
    Based on the title and abstract of the following research paper, evaluate its relevance to the topic 'linking environmental changes to health' on a scale from 0 (no relevance) to 10 (highly relevant). Please provide only a single number (from 0 to 10) as your response without any additional explanation.
    Title: {title}
    Abstract: {abstract}
    """

    client = OpenAI(
    organization=organization,
    project=project,
    api_key = api_key
    )
    completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": "You are a research assistant."},
        {"role": "user", "content": prompt}
    ],
        max_tokens=10,  # output tokens cap
        temperature=0.2,  # low temp values means more deterministic
        n=2, # generates two responses to the prompt
        stop=["\n"] # stop generating at new line
    )
    
    return completion

In [25]:
df['response_0'] = None
df['response_1'] = None

In [31]:
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    title = row['title']
    abstract = row['abstract']
    response = generate_response(title, abstract) # currently we make api calls for every row of the data frame
    df.at[index, 'response_0'] = response.choices[0].message.content
    df.at[index, 'response_1'] = response.choices[1].message.content

  2%|▏         | 44/2933 [00:25<27:57,  1.72it/s] 


KeyboardInterrupt: 

In [18]:
# environmental effects on health

completion = generate_response(df.iloc[830].title, df.iloc[830].abstract)

print(completion.choices[0].message)


ChatCompletionMessage(content='9', role='assistant', function_call=None, tool_calls=None, refusal=None)


In [20]:
[completion.choices[0].message.content, completion.choices[1].message.content]

['9', '9']

{'custom_id': 'request-1', 'method': 'POST', 'url': '/v1/chat/completions', 'body': {'model': 'gpt-3.5-turbo-0125', 'messages': [{'role': 'system', 'content': 'You are a helpful research assistant.'}, {'role': 'user', 'content': " Based on the title and abstract of the following research paper, evaluate its relevance to the topic 'linking environmental changes to health' on a scale from 0 (no relevance) to 10 (highly relevant). Please provide only a single number (from 0 to 10) as your response without any additional explanation.            Title: Dynamic Spending Responses to Wealth Shocks: Evidence from Quasi Lotteries on the Stock Market            Abstract: How much and over what horizon do households adjust their consumption in response to stock market wealth shocks? We address these questions using granular data on spending and stock portfolios from a large bank and exploiting lottery-like variation in gains across investors with similar portfolio characteristics. Consistent with